In [1]:
import os
import math
import tensorflow as tf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from tensorflow.keras.preprocessing.image import img_to_array


In [2]:
dataset="dataset"
file   ="validation"
out="TFrecords_DS"
BATCH_SIZE = 32
IMG_SIZE = [160, 160]
class_names=["cats",'dogs']

In [3]:
output_directory=os.path.join(out,file )

PATH = os.path.join(dataset)
PATH = os.path.join(PATH, file+"/*/*")


In [4]:
list_ds = tf.data.Dataset.list_files(PATH)

totalims=tf.data.experimental.cardinality(list_ds).numpy()
print(type(list_ds))
print(len(list_ds))
lotes= math.ceil(totalims/BATCH_SIZE)
print(lotes)

<class 'tensorflow.python.data.ops.dataset_ops.ShuffleDataset'>
1000
32


In [5]:
def parse_image(filename):
    parts = tf.strings.split(filename, os.sep)
    label = parts[-2]
    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image)
    image = tf.image.resize(image, IMG_SIZE)      
    return image, label

In [6]:
DataSet = list_ds.map(parse_image)
DataSet = DataSet.shuffle(BATCH_SIZE*4)
DataSet = DataSet.repeat()
DataSet = DataSet.batch(BATCH_SIZE)

#totalims=tf.data.experimental.cardinality(DataSet).numpy()

3
print("{} imagenes en {} lotes de {}".format(totalims, lotes, BATCH_SIZE))

1000 imagenes en 32 lotes de 32


In [7]:
bar=widgets.FloatProgress(
    value=0,
    min=0,
    max=lotes,
    step=0.1,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)

In [8]:
#defeniciones para la serializacion de datos

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    # Create a dictionary with features that may be relevant.

def image_example(image_string, label):
    image_shape = tf.image.decode_jpeg(image_string).shape
    #print("image_shape",image_shape )
    
    feature = {
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [9]:
display(bar)

FloatProgress(value=0.0, bar_style='info', description='Loading:', max=32.0)

In [10]:
z=0
for x, batch in enumerate(DataSet.take(lotes)):
    images, labels =batch          
    
    paso= (x+1)*(len(images))
    bar.value=x
    
    output_filename = '%s-%.5d-of-%.5d' % (file, z, paso)
    output_file = os.path.join(output_directory, output_filename+".tfrecords")
      
    os.makedirs(output_directory, exist_ok=True)
    
    
   # print("imagen {} a {} de {}, tam lote:{}, #lote:{}".format(z, paso,totalims ,len(images),x  ) )
    #print("tam lote:{},  numero de lote :{}".format( )) 
    #print("output_file ",output_file) 
    
    with tf.io.TFRecordWriter(output_file) as writer:       
        for i in range(len(images)): 
            z+=1          
            img=img_to_array(images[i])
            image_raw= tf.io.encode_jpeg(img)
            #image_raw=img
            label=class_names.index(labels[i])
            tf_example = image_example(image_raw, label)
            writer.write(tf_example.SerializeToString())
        writer.close()
        
print("info: Con {} imagenes, se crearon {} lotes, con {} imagenes cada uno  ".format( z, lotes, BATCH_SIZE))  
print("info: dir: ",output_directory)  

info: Con 1024 imagenes, se crearon 32 lotes, con 32 imagenes cada uno  
info: dir:  TFrecords_DS/validation
